# Installing packages

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio "trl==0.14.0"
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip uninstall -y trl
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

# Import Libs

In [2]:
import numpy as np
import pandas as pd
import json

import torch
import torch.nn.functional as F
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset

import os
from tqdm.auto import tqdm
from IPython.display import clear_output

os.environ["WANDB_DISABLED"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-25 16:27:47.638021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750868867.854676      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750868867.910909      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


# Initializing Model

In [3]:
model_name = "unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit"

# >>> Download LLM Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 8192,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model)
print("Model is ready to inference")

==((====))==  Unsloth 2025.6.5: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Model is ready to inference


# Load & process dataset

In [4]:
ds = load_dataset("nhantruongcse/summary-vietnamese-news")["train"]

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/165M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/164573 [00:00<?, ? examples/s]

In [5]:
system_message = """Bạn là trợ lý AI có nhiệm vụ tóm tắt văn bản của người dùng cung cấp.

### Hướng dẫn trả lời:
Không dài dòng, giới hạn 100 từ
Không giải thích gì thêm, chỉ cần tóm tắt
"""

user_template = """Hãy tóm tắt văn bản dưới đây:

{}
"""

assistant_template = """{}
"""

def apply_template_message(data):
    messages = [{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": user_template.format(data["Content"])
    }, {
        "role": "assistant",
        "content": assistant_template.format(data["Summary"])
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False, enable_thinking=False)
    data["text"] = text
    return data

def apply_template_message_eval(data):
    messages = [{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": user_template.format(data["Content"])
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, e=False)
    data["text"] = text
    return data

In [6]:
split_dataset = ds.train_test_split(test_size=0.2, seed=42)

train_ds = split_dataset["train"].select(range(10000)) # Just take 10.000 samples
test_ds = split_dataset["test"].select(range(2000)) # Just take 2.000

In [7]:
train_ds = train_ds.map(apply_template_message)
test_ds = test_ds.map(apply_template_message_eval)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Inference

In [8]:
def generate(prompt:str, max_new_tokens=200):
    inputs = tokenizer(
        [prompt], return_tensors="pt"
    ).to("cuda")
    input_length = inputs.input_ids.shape[1]  # Lưu độ dài ban đầu
        
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id, use_cache=True)
    new_tokens = output_ids[0, input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    return response.strip()

In [9]:
data_dict = {
    "Content": [],
    "Summary": [],
    "Pretrain_summary": []
}

for data in tqdm(test_ds, desc="Inferencing"):
    data_dict["Content"].append(data["Content"])
    data_dict["Summary"].append(data["Summary"])
    data_dict["Pretrain_summary"].append(generate(data["text"]))

Inferencing:   0%|          | 0/2000 [00:00<?, ?it/s]

# Save results

In [10]:
result_df = pd.DataFrame(data_dict)
result_df.head()

,Content,Summary,Pretrain_summary
0,Huỳnh Như (9) và Tuyết Dung (7) là cả kho tàng...,Ở sân chơi đẳng cấp nhất thế giới như World Cu...,Huỳnh Như và Tuyết Dung là cả kho tàng kinh ng...
1,Chiến công mới của các anh công an thật tuyệt ...,Việc công an nhanh chóng bắt được 2 nghi phạm ...,"Trong ngày 27/5, các lực lượng công an đã thực..."
2,"""Tôi rất vui và phấn khởi, cũng có đôi chút hồ...","Ngay ở lần tham dự Á vận hội đầu tiên, đội tu...","Trần Nguyên Hưng, VĐV đội tuyển Liên Minh Huyề..."
3,"Sau hai tháng liên tiếp sụt giảm, mới đây sức ...",Sức mua ô tô trên thị trường vừa có dấu hiệu h...,"Đầu tháng 10/2023, thị trường ô tô Việt Nam đã..."
4,"Theo đó, mức lãi suất cho vay ưu đãi tại Ngân ...",Thủ tướng đã ký Quyết định 486 về mức lãi suất...,Dưới đây là cách tóm tắt văn bản của bạn:\n\n-...


In [11]:
result_df.to_csv("pretrain_result.csv", index=False)